- [참고 data2vec audio:사용 안하기로함]: https://towhee.io/audio-embedding/data2vec
- [참고 data2vec text]: https://huggingface.co/docs/datasets/nlp_load
- [참고 data2vec load audio*]:https://huggingface.co/docs/datasets/audio_load
- [참고 data2vec embedding]: https://huggingface.co/docs/transformers/model_doc/data2vec#transformers.Data2VecAudioModel

In [58]:
import torch
from transformers import Wav2Vec2Processor, Data2VecAudioModel
from datasets import load_dataset
from scipy.io import wavfile # to be added
from glob import glob

processor = Wav2Vec2Processor.from_pretrained("facebook/data2vec-audio-base-960h")
model = Data2VecAudioModel.from_pretrained("facebook/data2vec-audio-base-960h")


Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [59]:
# from towhee import pipe, ops, DataCollection # to be added
# p = (pipe.input('path')
#         .map('path', 'frame', ops.audio_decode.ffmpeg(sample_rate=16000))
#         .map('frame', 'vecs', ops.audio_embedding.data2vec(model_name='facebook/data2vec-audio-base-960h'))
#         .output('path', 'vecs'))

# DataCollection(p('./org_KEMDy20/Session01/Sess01_script01_User001F_001.wav')).show()

In [60]:
wav_file_lst = glob('./org_KEMDy20/Session01/*.wav')
wav_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script03_User002M_010.wav',
 './org_KEMDy20/Session01/Sess01_script03_User001F_014.wav',
 './org_KEMDy20/Session01/Sess01_script05_User002M_001.wav']

In [61]:
from datasets import Dataset, Audio

dataset = Dataset.from_dict({"data": wav_file_lst }).cast_column("data", Audio())
dataset[0]["data"]

{'path': './org_KEMDy20/Session01/Sess01_script03_User002M_010.wav',
 'array': array([-0.00018311,  0.00057983,  0.00076294, ...,  0.00259399,
         0.00115967,  0.00036621], dtype=float32),
 'sampling_rate': 16000}

In [62]:
len(dataset)


311

In [63]:
from transformers import AutoFeatureExtractor, Data2VecAudioForSequenceClassification
from datasets import load_dataset
import torch

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/data2vec-audio-base-960h")
model = Data2VecAudioForSequenceClassification.from_pretrained("facebook/data2vec-audio-base-960h")

Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioForSequenceClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Data2VecAudioForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Data2VecAudioForSequenceClassification were not initialized from the model checkpoint at facebook/data2vec-audio-base-960h and are newly initialized: ['classifier.bias', 'projector.bias', 'classifier.weight', 'projector.weight']
You should probably TRAIN this model on a down-stream task to b

- dataset에 데이터 더하는 방법 찾아보기

In [71]:
# audio file is decoded on the fly
for i in range(len(dataset)):
    input = feature_extractor(dataset[i]["data"]["array"], sampling_rate=dataset[0]['data']['sampling_rate'], return_tensors="pt")
    # print(input['input_values'][0])
    # if 'embedded_audio_data' not in dataset[i]['data']:
    #     dataset[i]['data']['embedded_audio_data'] = None
    dataset[i]['data']['embedded_audio_data'] = input['input_values'][0]

In [74]:
# dataset[1]['data']['embedded_audio_data']
print(type(dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [66]:
txt_file_lst = glob('./org_KEMDy20/Session01/*.txt')
txt_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script06_User002M_009.txt',
 './org_KEMDy20/Session01/Sess01_script01_User001F_007.txt',
 './org_KEMDy20/Session01/Sess01_script01_User001F_001.txt']

In [67]:
# 참고: https://huggingface.co/docs/datasets/nlp_load
# text_dataset = load_dataset("text", data_files={"train": ["my_text_1.txt", "my_text_2.txt"], "test": "my_test_file.txt"})
text_dataset = load_dataset("text", data_files={"train": txt_file_lst[:int(0.8*len(txt_file_lst))], "test": txt_file_lst[int(0.8*len(txt_file_lst)):]})
text_dataset

Resolving data files: 100%|██████████| 63/63 [00:00<00:00, 127776.19it/s]
2023-03-21 18:02:04,648 - 139945135015744 - builder.py-builder:798 - WARNING: Found cached dataset text (/home/arplab/.cache/huggingface/datasets/text/default-92102c17bfb54e88/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
100%|██████████| 2/2 [00:00<00:00, 201.80it/s]


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 248
    })
    test: Dataset({
        features: ['text'],
        num_rows: 63
    })
})